In [70]:
import pandas as pd
from datetime import timedelta, datetime
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [3]:
users_df = pd.read_csv("takehome_users.csv", encoding="latin-1")
user_engagement_df = pd.read_csv("takehome_user_engagement.csv", encoding="latin-1")

In [3]:
users_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [16]:
user_engagement_df.date.min()

datetime.date(2012, 5, 31)

In [6]:
len(user_engagement_df[user_engagement_df.visited == 1])

207917

In [4]:
user_engagement_df["date"] = pd.to_datetime(user_engagement_df.time_stamp).dt.date

In [14]:
user_engagement_df.groupby(["user_id", "date"]).size().reset_index().rename(columns={0:'count'})

,user_id,date,count
0,1,2014-04-22,1
1,2,2013-11-15,1
2,2,2013-11-29,1
3,2,2013-12-09,1
4,2,2013-12-25,1
...,...,...,...
207912,11996,2013-09-06,1
207913,11997,2013-01-15,1
207914,11998,2014-04-27,1
207915,11999,2012-06-02,1


In [25]:
user_engagement_df

,date,time_stamp,user_id,visited
0,2014-04-22,2014-04-22 03:53:30,1,1
1,2013-11-15,2013-11-15 03:45:04,2,1
2,2013-11-29,2013-11-29 03:45:04,2,1
3,2013-12-09,2013-12-09 03:45:04,2,1
4,2013-12-25,2013-12-25 03:45:04,2,1
...,...,...,...,...
207912,2013-09-06,2013-09-06 06:14:15,11996,1
207913,2013-01-15,2013-01-15 18:28:37,11997,1
207914,2014-04-27,2014-04-27 12:45:16,11998,1
207915,2012-06-02,2012-06-02 11:55:59,11999,1


In [5]:
user_engagement_df = user_engagement_df.sort_values(["user_id", "date"], axis=0)

In [27]:
user_engagement_df

,date,time_stamp,user_id,visited
0,2014-04-22,2014-04-22 03:53:30,1,1
1,2013-11-15,2013-11-15 03:45:04,2,1
2,2013-11-29,2013-11-29 03:45:04,2,1
3,2013-12-09,2013-12-09 03:45:04,2,1
4,2013-12-25,2013-12-25 03:45:04,2,1
...,...,...,...,...
207912,2013-09-06,2013-09-06 06:14:15,11996,1
207913,2013-01-15,2013-01-15 18:28:37,11997,1
207914,2014-04-27,2014-04-27 12:45:16,11998,1
207915,2012-06-02,2012-06-02 11:55:59,11999,1


In [6]:
adopted_user_dict = {}
cur_user_id = 0
cur_logging_count = 0

for index, row in user_engagement_df.iterrows():
    if row["user_id"] != cur_user_id:
        window = []
        skip_user = False
        cur_user_id = row["user_id"]
        adopted_user_dict[cur_user_id] = False
        cur_logging_count = 1
        window.append(row["date"])
    else:  # same id, continue the window
        if not skip_user:
            while (len(window) > 0) and (row["date"] >= (window[0] + timedelta(days=7))):
                window.pop(0)
                cur_logging_count -= 1
            window.append(row["date"])
            cur_logging_count += 1
            if cur_logging_count >= 3:
                adopted_user_dict[cur_user_id] = True
                skip_user = True

In [42]:
user_engagement_df[user_engagement_df.user_id == 42]

,date,time_stamp,user_id,visited
349,2012-11-13,2012-11-13 19:05:07,42,1
350,2012-12-16,2012-12-16 19:05:07,42,1
351,2012-12-18,2012-12-18 19:05:07,42,1
352,2012-12-24,2012-12-24 19:05:07,42,1
353,2012-12-25,2012-12-25 19:05:07,42,1
...,...,...,...,...
686,2014-05-12,2014-05-12 19:05:07,42,1
687,2014-05-18,2014-05-18 19:05:07,42,1
688,2014-05-21,2014-05-21 19:05:07,42,1
689,2014-05-23,2014-05-23 19:05:07,42,1


In [33]:
adopted_user_dict

{1: False,
 2: True,
 3: False,
 4: False,
 5: False,
 6: False,
 7: False,
 10: True,
 11: False,
 13: False,
 14: False,
 17: False,
 19: False,
 20: True,
 21: False,
 22: False,
 23: False,
 24: False,
 25: False,
 27: False,
 28: False,
 29: False,
 30: False,
 31: False,
 33: True,
 36: False,
 37: False,
 41: False,
 42: True,
 43: True,
 44: False,
 45: False,
 46: False,
 47: False,
 48: False,
 49: False,
 50: True,
 51: False,
 53: True,
 54: False,
 55: False,
 56: False,
 57: False,
 58: False,
 59: False,
 60: False,
 61: False,
 63: True,
 64: False,
 65: False,
 66: False,
 67: False,
 68: False,
 69: True,
 72: False,
 73: False,
 74: True,
 75: False,
 76: False,
 77: False,
 78: False,
 80: True,
 81: True,
 82: True,
 83: False,
 84: False,
 85: False,
 86: False,
 87: True,
 88: False,
 89: False,
 90: False,
 91: False,
 92: False,
 94: False,
 95: False,
 96: False,
 97: False,
 98: False,
 99: False,
 100: False,
 101: False,
 103: False,
 105: False,
 106: Fals

In [7]:
users_df = users_df.set_index("object_id")

In [44]:
users_df.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [8]:
users_df["adopted"] = pd.Series(adopted_user_dict)

In [9]:
users_df["adopted"] = users_df["adopted"].fillna(False)

In [10]:
len(users_df["org_id"].value_counts())

417

In [11]:
len(users_df[users_df["adopted"]])

1602

In [12]:
users_df["email_domain"]=users_df['email'].str.extract(r'.+@(.+)\.com')

In [21]:
def EmailGroup(row):
    if row["email_domain"] in ["gmail", "yahoo", "jourrapide", "gustr", "hotmail"]:
        return row["email_domain"]
    else:
        return "other"

In [22]:
users_df["email_group"] = users_df.apply(lambda row: EmailGroup(row), axis=1)

In [23]:
users_df.creation_time

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted,email_domain,email_group
object_id,,,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False,yahoo,yahoo
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True,gustr,gustr
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False,gustr,gustr
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False,yahoo,yahoo
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False,yahoo,yahoo
...,...,...,...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0,False,gustr,gustr
11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN,False,gmail,gmail
11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0,False,NaN,other


In [31]:
users_df["creation_time_seconds"] = (pd.to_datetime(users_df['creation_time']) - datetime(2012,1,1)).dt.total_seconds()

In [32]:
users_df

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted,email_domain,email_group,creation_time_seconds
object_id,,,,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False,yahoo,yahoo,72762810.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True,gustr,gustr,59111104.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False,gustr,gustr,38358892.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False,yahoo,yahoo,43747768.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False,yahoo,yahoo,33041660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0,False,gustr,gustr,53072055.0
11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN,False,gmail,gmail,32466517.0
11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0,False,NaN,other,73226716.0


In [37]:
users_df = pd.get_dummies(users_df, columns=["creation_source", "email_group"])

In [41]:
users_df = users_df.drop(["creation_time", "name", "email", "org_id", "invited_by_user_id", "email_domain", "creation_source_SIGNUP_GOOGLE_AUTH", "email_group_other"], axis=1)

In [84]:
X = users_df.drop("adopted", axis=1)

In [45]:
y = users_df["adopted"]

In [87]:
X["last_session_creation_time"] = X["last_session_creation_time"].fillna(X["last_session_creation_time"].mean())

In [88]:
X["last_session_creation_time"] = X["last_session_creation_time"] / 1000000000

In [89]:
X[["last_session_creation_time", "creation_time_seconds"]] = preprocessing.scale(X[["last_session_creation_time", "creation_time_seconds"]])

In [91]:
clf = LogisticRegression(penalty="l1", solver="liblinear").fit(X, y)

In [94]:
X.columns

Index(['last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'creation_time_seconds',
       'creation_source_GUEST_INVITE', 'creation_source_ORG_INVITE',
       'creation_source_PERSONAL_PROJECTS', 'creation_source_SIGNUP',
       'email_group_gmail', 'email_group_gustr', 'email_group_hotmail',
       'email_group_jourrapide', 'email_group_yahoo'],
      dtype='object')

In [93]:
clf.coef_

array([[ 3.55311128, -0.02231253,  0.        , -1.9507944 , -0.00567585,
        -0.17317346, -1.09447556, -0.03251224,  0.21536975, -0.10432004,
         0.22365619, -0.0467328 , -0.14457132]])